# 과제2_Part1_120220243_황상민

# ❗ Caution ❗

이 코드는 특정 웹페이지에 대한 크롤링을 포함하고 있습니다.<br>
작성자는 크롤러에 적절한 time.sleep을 적용하지 않고 크롤러를 작성하여 쿠팡으로부터 한번에 크롤링이 되지 않도록 IP를 차단당하였습니다.<br>
그래서 혹시 교수님이나 조교님, 혹은 다른 학생분들께서 이 코드를 replicate하고자 하신다면, 코드 내에 충분한 수치로 부여된 time.sleep()을 임의로 줄이지 마시길 바랍니다.<br>
그러지 않을 경우에 추후 본인의 IP로 쿠팡 웹페이지를 크롤링하는데 어려움이 있을 수 있습니다. 그리고 Par1 크롤러에 포함된 request library 때문에 google colab 환경에서는 실행하기 어려우므로 local 환경에서 실행해야 합니다.<br>
Part1에는 데이터의 수집과정이 담겨있고, 과제의 결과만 확인하고 싶으시다면 Part2 코드를 참고하시길 바랍니다.<br><br>

# ❓ Question ❓
# 상품의 이미지와 e커머스 플랫폼 검색결과 페이지 번호와의 상관관계
### 쿠팡에서 세제 키워드를 중심으로

### Motivation<br>
요즘 신제품을 출시하면 e커머스에 제품을 등록하는 것은 불가피하다. e커머스  플랫폼에서 사용자는 필터기능보다는 검색기능을 주로 활용한다. 검색은 키워드를 통해 이뤄지는데, 노출되는 수많은 제품 리스트는 한 화면에 표시되기에는 제한된다. 그래서 많은 e커머스 플랫폼은 여러 페이지에 제품을 노출한다. 상위 페이지에 노출될수록 소비자에게 많이 노출되어 매출을 높일 수 있다. 그러나 수많은 판매자의 수많은 제품을 e커머스 플랫폼에서 어떻게 배치할까?<br>
제품포장에서 시각이미지는 언어적이거나 비언어적으로 소비자의 이성적 또는 감성적으로 구매동기를 유발하고 촉진시키는 핵심적인 요소이자 상품의 경쟁력 강화를 위한 마케팅 커뮤니케이션 수단이 되고 있다.¹ 그래서 이번 과제에서는 e커머스 플랫폼 웹페이지에서 소비자가 키워드로 제품을 검색했을 때, 소비자에게 노출되는 상품의 이미지가 상품이 노출되는 페이지의 번호와 얼마나 강한 상관관계가 있는지 확인하고자 한다. 이를 실험하기 위해 쿠팡에서 주요 생필품 중 하나인 세제를 검색했을 때, 각 세제 상품이 노출되는 페이지 번호를 예측하는 CNN모델을 구축한다. 이를 통해 상품의 이미지와 페이지 번호 간의 상관관계 추정하고, 신제품의 이미지로 온라인 쇼핑몰에 진입했을 때 위치할 페이지 번호를 예측하고자 한다.
>¹김경선 ( Kim Kyung-sun ),and 강혜숙 ( Kang Hye-sook ). "제품포장의 시각이미지에서 인지반응과 감성반응이 제품태도와 구매의도에 미치는 영향." 한국디자인포럼 28.- (2010): 197-210.
<br><br>


### Experiment Environment<br>
쿠팡은 모바일 애플리케이션과 웹페이지로 플랫폼을 운영하고 있다. 이번 과제에서는 웹페이지의 검색결과를 중심으로 노출되는 페이지 번호를 예측하고자 한다. 검색결과 노출은 '쿠팡 랭킹순'이라는 기본 기준을 적용하여 노출되며 페이지당 36개의 상품이 노출된다. 쿠팡 랭킹순은 판매 실적, 사용자 선호도, 상품 정보 충실도 및 검색 정확도 등을 종합적으로 고려한 순위다. 즉, 이번 과제에서는 쿠팡에서 세제를 검색했을 때, 쿠팡 랭킹순으로 노출될 때의 페이지 번호를 이미지만으로 예측하는 실험을 수행한다.<br>
과제의 코드는 크게 두 파트로 나뉜다. 첫번째 파트는 크롤링을 통해 세제 상품정보 및 이미지를 수집하는 코드이고, 두번째 파트는 수집된 데이터를 이용해 예측을 수행하는 코드다. 첫번째 파트인 크롤링 코드는 Google Colab에서 실행할시, request library가 올바르게 작동하지 않아서 local 환경에서 replicator가 지정한 작업 디렉토리에서 수집된 데이터를 저장해야 한다. 두번째 파트는 작성자의 dropbox link를 통해 제공되는 미리 수집된 데이터를 사용하여 모델링 과정을 Google Colab 환경에서도 수행해볼 수 있다.
<br><br>

### Issue & Applicable Solution

이번 과제에서 효용대비 비용문제를 고려하여 해결하지 못했지만 솔루션이 존재하는 이슈가 있다.

1) 같은 이미지를 사용하는 상품을 다른 판매자가 올린 경우<br>
-> 이미지 유사도를 검사하며 100% 일치하는 이미지의 경우, 상위 페이지에 위치한 이미지 삭제<br>
-> 새로 입점하는 상품은 하위 페이지에 위치할 가능성이 높기 때문에 상위 페이지 이미지를 삭제<br><br>
2) 같은 페이지 내에서도 상, 하단 순위 부여 부재<br>
-> 쿠팡 웹페이지 html tag 활용하여 Xpage의 Y번째 행에 위치한 상품의 페이지 번호를 X.(Y-1)로 부여(한 페이지에 10행의 이미지로 구성)<br>
-> e.g. 5페이지의 7번째 행에 위치한 상품은 5.6페이지로 페이지 번호 부여
<br><br>





### Expandability<br>
이번 과제의 주제를 추후 팀 프로젝트 범위로 확장할 수 있는 방향을 제시한다.<br>
1) 여러 품목의 이미지와 페이지 번호 상관관계 비교<br>
여러 품목에서 모델의 페이지 번호 예측 성능을 비교함으로써 상품의 이미자와 페이지 번호의 상관관계가 강한 품목과 약한 품목을 추려낸다. 이를 바탕으로 e커머스 플랫폼은 페이지 노출 방식을 수정할 수 있다. 그리고 입점 업체는 상품 이미지와 페이지 번호의 상관관계에 따라 노출되는 상품의 어떤 정보를 수정해야 상위 페이지에 노출되는지 판단할 수 있다.<br>

2) 특정 품목에 대한 여러 e커머스 플랫폼 페이지 번호 부여경향 비교<br>
특정 품목을 기준으로 서로 다른 e커머스 플랫폼에서 상품의 이미지와 페이지 번호의 상관관계를 파악한다. 이 경우에는 각 e커머스 플랫폼마다 다른 웹페이지 구조를 갖고있기 때문에 각각의 크롤러를 생성해야 할 것이다. 그리고 한 페이지에 배치된 상품 개수 및 키워드 당 최대 페이지 개수 등과 같은 여러 요소를 고려하여 데이터 수집이 이루어져야 한다.
<br><br>

# 1. Import libraries

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import time
import math

#change path
#os.chdir('C:/Users/sangmin/OneDrive - 아주대학교/onedrive_pypr/경제정보처리/과제2')

#os.chdir('INPUT YOUR LOCAL PATH') # request library의 문제로 인하여 코랩에서 part1 실행이 어려움

# 2. Define Functions

## 2.1. 검색된 키워드 마지막 페이지 번호 추출

쿠팡의 검색결과 URL은 <br>'https://www.coupang.com/np/search?q={keyword}&channel=user&sorter=scoreDesc&listSize=36&isPriceRange=false&rating=0&page={page}&rocketAll=false'<br>
이런 구조로 구성돼 있다.

검색의 키워드와 페이지 번호만 있으면 해당하는 상품들을 확인할 수 있다.

이번 과제에서는 세제를 검색했을 때, 1페이지부터 끝페이지까지 있는 상품의 이미지와 페이지 번호를 수집할 필요가 있다.

이를 위해서 우선 쿠팡의 페이지 구조를 이해할 필요가 있다.

쿠팡의 경우에 키워드로 검색하여 노출되는 페이지가 최대 27페이지까지 있다.<br>
인기가 없어서 등록된 상품의 개수가 적은 키워드의 경우에는 끝페이지가 27페이지가 아닐수도 있다.<br>
그래서 쿠팡의 HTML 구조에서 끝페이지의 정보를 담고 있는 태그를 추출하는 함수를 작성했다.

구글 크롬에서 F12를 눌러 개발자 도구를 이용하면 웹페이지에서 원하는 정보가 HTML 구조 어디에 위치한지 쉽게 찾을 수 있다.

In [ ]:
# 검색된 키워드의 마지막 페이지 번호를 추출하는  함수

def find_last_page(keyword):
    baseurl = 'https://www.coupang.com'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36' # 크롤러가 아니라 사람이라고 하는 부분
    }

    first_url = f'https://www.coupang.com/np/search?q={keyword}&channel=user&sorter=scoreDesc&listSize=36&isPriceRange=false&rating=0&page=1&rocketAll=false'
    response = requests.get(first_url, headers=headers)
    soup = BeautifulSoup(response.content, "lxml") #lxml을 사용해서 soup에 넣겠다.
    products_page = soup.find('a', class_='btn-last disabled').text # soup에서 a tag에서 class=btn-last disabled인것을 찾아라
    
    return int(products_page) # 키워드의 조회가능한 마지막 페이지 번호 반환

In [ ]:
find_last_page('세제') # 쿠팡에서 세제를 검색했을 때 조회할 수 있는 끝페이지

KeyboardInterrupt: ignored

## 2.2. 추출할 상품의 데이터 수집

code source: https://goodthings4me.tistory.com/562

In [ ]:
# code source: https://goodthings4me.tistory.com/562
# 긁어올 상품들의 정보 추출

def coupang_products(keyword):
    pages = find_last_page(keyword)-1 # 끝페이지는 비어있어서 크롤링 오류가 나는 경우가 있으므로, 뒤에서 두번재 페이지까지 사용한다.
    
    baseurl = 'https://www.coupang.com'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'
    }

    products_link = []
    file_number = 0
    for page in range(1, pages + 1): # 1 페이지부터 pages(맨 마지막에서 앞의 페이지)까지 있는 매물 다 긁어
        url =f'https://www.coupang.com/np/search?q={keyword}&channel=user&sorter=scoreDesc&listSize=36&isPriceRange=false&rating=0&page={page}&rocketAll=false'
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "lxml") #lxml을 사용해서 soup에 넣겠다
        products_lis = soup.find('ul', id='productList').find_all('li') # soup에서 ul tag에서 id=productList인거에서li'태그를 모두 가져온다.
        for li in products_lis: # 페이지 내에 있는 모든 product list를 확인하면서
            a_link = li.find('a', href=True)['href'] # li tag에서 a태그를 찾아서 href가 있는 tag를 찾아서 href를 a_link에 넣어라 
            prd_link = baseurl + a_link # base url이랑 a_link 이어붙여서 prd_link에 넣어라
            prd_name = li.find('div', class_='name').text # li tag에서 div tag가 class=name인 것을 텍스트로 변환해서 제품명으로 저장하라
            prd_page = page # 현재 긁고 있는 페이지를 상품 페이지에 저장하라
            try: # try code가 되면 try code를 실행하고
                base_price = li.find('span', class_='price-info').find('del', class_='base-price').text
            except: # try code가 실행되지 않으면 except 코드를 실행해라
                base_price = ''
            price = li.find('strong', class_='price-value').text

            file_number = file_number + 1 # 페이지 번호에 상관없이 수집하고 있는 파일에 임의로 부여하는 숫자
            file_number_str = str(file_number).zfill(6) # 임의로 부여된 숫자를 6자리 고정
            products_info = {
                'file_name': 'coupang_' + '%dpage_' %page + '%s_' %keyword + file_number_str + '.jpg', 
                'product_type': keyword,
                'product_name': prd_name,
                'base_price': base_price,
                'price': price,
                'product_page': prd_page,
                'product_url': prd_link
            }
            products_link.append(products_info)

            time.sleep(2)
        time.sleep(5)

    print(len(products_link))
    df = pd.DataFrame(products_link)
    print(df)
    df.to_csv('coupang_%s_%dpages_list.csv' %(keyword, pages), index=False, encoding='utf-8-sig')

In [ ]:
start = time.time() 

coupang_products('세제')

end = time.time() 
print(f"{end - start:.5f} sec")

## 2.3. 추출한 상품 데이터의 링크를 통해 이미지 추출 및 저장

code source: https://data-make.tistory.com/170

In [ ]:
# 추출한 이미지를 저장할 폴더를 생성하는 함수

def createFolder(directory):
    import os
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [ ]:
# 이미지 크롤링

def crawl_img_from_df(input_df) :   
    for index in range(0, len(input_df)) :       
        img_url = input_df.iloc[index,:]['product_url']
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
        res = requests.get(img_url, headers=headers)
        res.raise_for_status() # 200 OK 코드가 아닌 경우 에러 발동
        soup = BeautifulSoup(res.text, "lxml") 
        
        prd_img_url = "http:"+ soup.select_one('#repImageContainer > img')['src']
        prd_type = input_df.iloc[index,:]['product_type']
        prd_page = input_df.iloc[index,:]['product_page']
        img_name = input_df.iloc[index,:]['file_name']
        img_path1 = './%s_img/'%(prd_type)
        img_path2 = './%s_%dpage/'%(prd_type, prd_page)
        
        # 클래스별로 저장필요할 때 사용
        #createFolder('%s_%dpage' %(prd_type, prd_page))
        #urllib.request.urlretrieve(prd_img_url, img_path2 + "%s" %(img_name))
        
        # 그냥 한 폴더에 이미지 저장
        createFolder('%s_img' %(prd_type))
        urllib.request.urlretrieve(prd_img_url, img_path1 + "%s" %(img_name))
        time.sleep(5)

In [ ]:
start = time.time() 

df_detergent = pd.read_csv('coupang_세제_26pages_list.csv')
crawl_img_from_df(df_detergent)

# 이미지 저장에 얼마나 걸린지 확인
end = time.time() 
print(f"{end - start:.5f} sec")

Part1을 통해 수집한 상품 정보 및 이미지 데이터는 연구자가 직접 Dropbox로 옮겨 Part2에서 누구나 접근가능하도록 제공함